In [159]:
import logging
from typing import Dict, Tuple
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, roc_curve, precision_score
from sklearn.model_selection import train_test_split

In [4]:
data=pd.read_csv("Data.csv")

In [5]:
data["Sintoma"].nunique()

8303

In [6]:
data["Enfermedad"].nunique()

4262

In [7]:
data["Frecuencia"].nunique()

6

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112689 entries, 0 to 112688
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Enfermedad  112689 non-null  object
 1   Sintoma     112689 non-null  object
 2   Frecuencia  112689 non-null  object
dtypes: object(3)
memory usage: 2.6+ MB


In [9]:
data2=data.drop_duplicates()

In [10]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112657 entries, 0 to 112688
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Enfermedad  112657 non-null  object
 1   Sintoma     112657 non-null  object
 2   Frecuencia  112657 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


In [11]:
data3=data2.dropna()

In [12]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112657 entries, 0 to 112688
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Enfermedad  112657 non-null  object
 1   Sintoma     112657 non-null  object
 2   Frecuencia  112657 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


In [139]:
data4=data3[(data3['Frecuencia']=='Muy frecuente (99-80%)') | (data3['Frecuencia']=='Frecuente (79-30%)')] 
data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63624 entries, 0 to 112682
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Enfermedad  63624 non-null  object
 1   Sintoma     63624 non-null  object
 2   Frecuencia  63624 non-null  object
dtypes: object(3)
memory usage: 1.9+ MB


In [146]:
#data4 = data4.iloc[:2024]

#data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2024 entries, 0 to 3308
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Enfermedad  2024 non-null   object
 1   Sintoma     2024 non-null   object
 2   Frecuencia  2024 non-null   object
dtypes: object(3)
memory usage: 63.2+ KB


In [148]:
data4.tail(20)

,Enfermedad,Sintoma,Frecuencia
3289,Abetalipoproteinemia,Abnormal apolipoprotein level,Muy frecuente (99-80%)
3290,Abetalipoproteinemia,Low levels of vitamin E,Muy frecuente (99-80%)
3291,Abetalipoproteinemia,Progressive visual loss,Frecuente (79-30%)
3292,Abetalipoproteinemia,Color vision defect,Frecuente (79-30%)
3293,Abetalipoproteinemia,Nyctalopia,Frecuente (79-30%)
3294,Abetalipoproteinemia,Areflexia,Frecuente (79-30%)
3295,Abetalipoproteinemia,Failure to thrive,Frecuente (79-30%)
3296,Abetalipoproteinemia,Anemia,Frecuente (79-30%)
3297,Abetalipoproteinemia,Reticulocytosis,Frecuente (79-30%)
3298,Abetalipoproteinemia,Chronic diarrhea,Frecuente (79-30%)


In [149]:
enfermedades=data4.iloc[:,0]
enfermedades.info()


<class 'pandas.core.series.Series'>
Int64Index: 2024 entries, 0 to 3308
Series name: Enfermedad
Non-Null Count  Dtype 
--------------  ----- 
2024 non-null   object
dtypes: object(1)
memory usage: 31.6+ KB


In [150]:
enfermedades_sin_rep=enfermedades.drop_duplicates()
enfermedades_sin_rep.info()

<class 'pandas.core.series.Series'>
Int64Index: 92 entries, 0 to 3285
Series name: Enfermedad
Non-Null Count  Dtype 
--------------  ----- 
92 non-null     object
dtypes: object(1)
memory usage: 1.4+ KB


In [152]:
enfermedades_sin_rep.head(20)

0                                Enfermedad de Alexander
61                                      Alfa-manosidosis
104                               Aspartilglucosaminuria
146                   Deficiencia múltiple de sulfatasas
176                                     Beta-manosidosis
182                                Enfermedad de Canavan
200                                           Cistinosis
234                                          Fucosidosis
264    Enfermedad de almacenamiento de glucógeno por ...
271    Enfermedad de almacenamiento de glucógeno por ...
278                           Mucopolisacaridosis tipo 6
303                                    Sialidosis tipo 1
348                                Mucolipidosis tipo IV
372                               Mucolipidosis tipo III
395                               Enfermedad de Sandhoff
414                           Mucopolisacaridosis tipo 7
442              Ictiosis recesiva ligada al cromosoma X
449                            

In [153]:
series=data4.iloc[:,1]
series.info()

<class 'pandas.core.series.Series'>
Int64Index: 2024 entries, 0 to 3308
Series name: Sintoma
Non-Null Count  Dtype 
--------------  ----- 
2024 non-null   object
dtypes: object(1)
memory usage: 31.6+ KB


In [154]:
series_sin_repe=series.drop_duplicates()
series_sin_repe.info()

<class 'pandas.core.series.Series'>
Int64Index: 916 entries, 0 to 3307
Series name: Sintoma
Non-Null Count  Dtype 
--------------  ----- 
916 non-null    object
dtypes: object(1)
memory usage: 14.3+ KB


In [155]:
series_sin_repe.head(20)

0                       Macrocephaly
1            Intellectual disability
2                            Seizure
3                         Spasticity
4        Agenesis of corpus callosum
5                      Hyperreflexia
6                     Megalencephaly
7                  Failure to thrive
8                    Frontal bossing
9                Nausea and vomiting
10    Neurological speech impairment
11                            Clonus
12                   EEG abnormality
13                 Sleep disturbance
14                         Scoliosis
15           Abnormal pyramidal sign
16                        Large face
17       Abnormality of eye movement
18                            Ptosis
19                         Nystagmus
Name: Sintoma, dtype: object

In [156]:
len(enfermedades_sin_rep)

92

In [157]:
len(series_sin_repe)

916

In [158]:
%%time

df_train=pd.DataFrame()
count=0
count_enfermedades=0
repeticiones=50
z=0
matriz_enfermedades=[]
while (z<repeticiones):
    j=0
    while (j<len(enfermedades_sin_rep)):
        vector_enfermedad=[]
        i=0
        vector_enfermedad.append(enfermedades_sin_rep.iloc[j])
        vector_enfermedad.append(j)
     #   print(enfermedades_sin_rep.iloc[j])
        while (i<len(series_sin_repe)):
            if True in ((data4["Enfermedad"]==enfermedades_sin_rep.iloc[j])
                    & (data4["Sintoma"]==series_sin_repe.iloc[i])
                    & (data4["Frecuencia"]=="Muy frecuente (99-80%)")).values: 
                
                valor_aleatorio = random.random()
                if (valor_aleatorio>0.30):
                    vector_enfermedad.append(1)
                else:
                    vector_enfermedad.append(0)
              
            elif True in ((data4["Enfermedad"]==enfermedades_sin_rep.iloc[j])
                    & (data4["Sintoma"]==series_sin_repe.iloc[i])
                    & (data4["Frecuencia"]=="Frecuente (79-30%)")).values: 
                if (valor_aleatorio>0.70):
                    vector_enfermedad.append(1)
                    

                else:
                    vector_enfermedad.append(0)
                    
                
                
            else:                 
                vector_enfermedad.append(0)
            i=i+1
        matriz_enfermedades.append(vector_enfermedad)
      #  print(j)
        j=j+1
    z=z+1    
    print(z)

df_train=pd.DataFrame(matriz_enfermedades)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
CPU times: user 54min 31s, sys: 23.2 s, total: 54min 54s
Wall time: 54min 55s


In [160]:
set(vector_enfermedad)

{0, 1, 91, 'Abetalipoproteinemia'}

In [161]:
count

0

In [162]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Columns: 918 entries, 0 to 917
dtypes: int64(917), object(1)
memory usage: 32.2+ MB


In [163]:
df_train.head(30)

,0,1,2,3,4,5,6,7,8,9,...,908,909,910,911,912,913,914,915,916,917
0,Enfermedad de Alexander,0,1,1,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Alfa-manosidosis,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aspartilglucosaminuria,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Deficiencia múltiple de sulfatasas,3,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Beta-manosidosis,4,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Enfermedad de Canavan,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Cistinosis,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,Fucosidosis,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Enfermedad de almacenamiento de glucógeno por ...,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Enfermedad de almacenamiento de glucógeno por ...,9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
df_train = df_train.rename(columns={0:'Enfermedad', 1:'Id_Enfermedad'})

In [165]:
target=df_train["Id_Enfermedad"]

In [166]:
df_train=df_train.drop (['Enfermedad'], axis=1)
df_train.head(20)


,Id_Enfermedad,2,3,4,5,6,7,8,9,10,...,908,909,910,911,912,913,914,915,916,917
0,0,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [167]:
X_train, X_test, y_train, y_test = train_test_split(df_train,target)
rf= RandomForestClassifier()
rf.fit (X_train,y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier()

In [168]:
X_train

,Id_Enfermedad,2,3,4,5,6,7,8,9,10,...,908,909,910,911,912,913,914,915,916,917
0,0,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3125,89,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2014,82,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3878,14,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2992,48,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,34,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2293,85,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2496,12,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4012,56,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [169]:
X_test

,Id_Enfermedad,2,3,4,5,6,7,8,9,10,...,908,909,910,911,912,913,914,915,916,917
1364,76,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
481,21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4343,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3309,89,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2423,31,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2772,12,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
350,74,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3719,39,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [170]:
prediccion=rf.predict(X_test)
prediccion

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([76, 21, 19, ..., 74, 39, 34])

In [171]:
y_test, prediccion

(1364    76
 481     21
 4343    19
 3309    89
 2423    31
         ..
 663     19
 2772    12
 350     74
 3719    39
 3254    34
 Name: Id_Enfermedad, Length: 1150, dtype: int64,
 array([76, 21, 19, ..., 74, 39, 34]))

In [130]:
#from sklearn import metrics
#print ("Accurancy:", metrics.accuracy_score(y_test, prediccion))


Accurancy: 0.9809523809523809


In [186]:
def evaluate(classifier, data, target):
   


    #X_train, X_test, y_train, y_test = train_test_split(data,target)
    #classifier.fit(X_train,y_train)
    y_pred=classifier.predict(X_test)
    probs=classifier.predict_proba(X_test)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    pres= precision_score (y_test, y_pred, average='micro')
    f1 = f1_score(y_test, y_pred, average='micro')
    #auc = roc_auc_score(y_test, probs)
    print ("Accurancy: ", acc)
    print ("Precision: ", pres)
    print ("f1: ", f1_score)
    #print ("auc: ", auc)
    
   # fpr, tpr, threshols = roc_curve(y_test, probs)
   # plt.plot(fpr,tpr)
   # plt.xlabel('False positives rate')
   # plt.ylabel('True positives rate')
   # plt.gcf().set_size_inches(8,6)
    return  f1, acc, pres

In [187]:
evaluate(rf, df_train, target)

Accurancy:  0.9973913043478261
Precision:  0.9973913043478261
f1:  <function f1_score at 0x7fc6c9cc3d30>


/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


(0.9973913043478261, 0.9973913043478261, 0.9973913043478261)

In [334]:
#import numpy as np

#new = (data5.groupby("Enfermedad")
 #        .agg({"Sintoma": np.array, "Frecuencia": np.array})
  #       .reset_index()
  #       )

In [108]:
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.preprocessing import OneHotEncoder
#enc = OneHotEncoder(handle_unknown='ignore')
#encoded= enc.fit_transform(df_prueba2[["Sintomas_paciente"]])
#df_prueba2[enc.categories_[0]]=encoded.toarray() 


#### 